In [70]:
# Imports
# Core PyTorch for tensor operations
import torch

# our DDPM components
from Noise import NoiseScheduler
from model import U_net

# Utilities
from tqdm import tqdm
import torchvision
import os

In [ ]:
# Setup for image generation

# Select device: GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Configuration
# Must match the configuration used during training!

# Image dimensions
image_size = 28

# Number of diffusion timesteps (must match training configuration)
num_timesteps = 1000

# Initialize noise scheduler
# Recreate the same noise schedule used during training
noise_scheduler = NoiseScheduler(
    num_timesteps=num_timesteps, 
    beta_start=0.0001,
    beta_end=0.02,
    device=device
)

# Load trained model
# Create model architecture (same as training)
model = U_net(device, in_channels=1)

# Load pre-trained weights from checkpoint
model_path = os.path.join("Save_model", f"model_DDPM.pth")
model.load_state_dict(torch.load(model_path, map_location=device))

# Move model to device and set to evaluation mode
model.to(device)
model.eval()  # Disable dropout, batch norm updates, etc.

print("Model loaded and ready for generation!")

In [ ]:
# Image generation from random noise
# This is the core generative process: starting from pure noise,
# iteratively remove noise to create new images

# Initialization
# Start with pure random Gaussian noise
# Shape: (1, 1, 28, 28) = single grayscale 28x28 image
xt = torch.randn(1, 1, image_size, image_size).to(device)

# Reverse diffusion loop
# At each timestep, predict and remove noise
with torch.no_grad():
    for i in tqdm(reversed(range(num_timesteps)), total=num_timesteps, desc="Generating image"):
        # Noise prediction
        # Give current noisy image and timestep to the trained model
        noise_pred = model(xt, torch.as_tensor(i).unsqueeze(0).to(device))

        # Denoising step
        # Remove predicted noise using reverse diffusion formula
        xt = noise_scheduler.sample_prev_timestep(xt, noise_pred, torch.as_tensor(i).to(device))

        # Image procesing
        img = torch.clamp(xt, -1., 1.).detach().cpu()
        
        # Denormalize: convert from [-1, 1] back to [0, 1] for display
        img = img * 0.5 + 0.5

        # Saving denoising progression
        # Save image at each denoising step to visualize the process

        # Convert PyTorch tensor to PIL Image
        to_pil = torchvision.transforms.ToPILImage()
        img_pil = to_pil(img[0])

        # Create output directory if needed
        os.makedirs("path/to/save", exist_ok=True)
        
        # Build file path:
        save_path = os.path.join(os.getcwd(), "path/to/save", f"x_{i}.png")

        # Save image to disk
        img_pil.save(save_path)
        img_pil.close()

print("Image generation complete!")